In [27]:
!pip install stop_words

In [28]:
!pip install pymorphy2

In [29]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATA/Lesson8NLP/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATA/Lesson8NLP/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATA/Lesson8NLP/val.csv")

In [34]:
df_train

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1
...,...,...,...
181462,181462,Классным будет новый год..( http://t.co/LejaGu...,0
181463,181463,Видишь человека хочешь сказать ему привет . А ...,0
181464,181464,@Julia6_9Styles там длинный диагноз. Вкратце: ...,0
181465,181465,"@technoslav ух ты, а кому тот серенький кнопоч...",1


In [35]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [36]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [37]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [38]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [39]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [69]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 50s 149ms/step - loss: 0.6229 - accuracy: 0.6248 - val_loss: 0.4927 - val_accuracy: 0.7520


In [71]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.5115 - accuracy: 0.7410


Test score: 0.5114575624465942
Test accuracy: 0.7409513592720032


In [76]:
model_conv = Sequential()
model_conv.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_conv.add(Conv1D(30, 3))
model_conv.add(Activation("relu"))
model_conv.add(GlobalMaxPool1D())
model_conv.add(Dense(10))
model_conv.add(Activation("relu"))
model_conv.add(Dense(1))
model_conv.add(Activation('softmax'))

In [77]:
model_conv.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [78]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_conv.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
319/319 [==============================] - 32s 97ms/step - loss: 0.6326 - accuracy: 0.5072 - val_loss: 0.4973 - val_accuracy: 0.5049


In [79]:
score = model_conv.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 4ms/step - loss: 0.5168 - accuracy: 0.5047


Test score: 0.5167723298072815
Test accuracy: 0.5047392249107361


In [80]:
model_lstm = Sequential()

model_lstm.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_lstm.add(Masking(mask_value=0.0))
model_lstm.add(LSTM(64, recurrent_dropout=0.2))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model_lstm.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 94s 275ms/step - loss: 0.6102 - accuracy: 0.6493 - val_loss: 0.4944 - val_accuracy: 0.7554


In [82]:
score = model_lstm.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 18ms/step - loss: 0.5056 - accuracy: 0.7468


Test score: 0.5056254863739014
Test accuracy: 0.7467706799507141


В итоге LSTM справился лучше других.

3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)

Вот третий пункт не совсем понимаю как сделать.